In [1]:
import json

with open('1_scrapping_black_clover.json', 'r') as openfile:
    eps_summary = json.load(openfile)

with open('3_characters_black_clover.json', 'r') as openfile:
    characters = json.load(openfile)

with open('3_overused_verbs_black_clover.json', 'r') as openfile:
    overused_verbs = json.load(openfile)

In [2]:
overused_verbs_lowered = []
for verb in overused_verbs:
    overused_verbs_lowered.append(verb)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /home/pauli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pauli/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/pauli/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [4]:
from nltk import word_tokenize
from nltk import sent_tokenize

In [5]:
tokens      = set()
sentences   = []
for ep_summary in eps_summary:
    sents = sent_tokenize(ep_summary)
    sentences.extend(sents)
    for sent in sents:
        words = word_tokenize(sent)
        for word in words:
            if not word.lower() in overused_verbs_lowered:
                tokens.add(word.lower())

In [6]:
token_to_index = dict()
index_to_token = dict()
i = 0
for token in tokens:
    token_to_index[token] = i
    index_to_token[i] = token
    i += 1

In [7]:
character_to_index = dict()
index_to_character = dict()
i = 0
for character in characters:
    character_to_index[character.lower()] = i
    index_to_character[i] = character.lower()
    i += 1

In [8]:
import numpy as np

In [9]:
def get_subjects_in_sent(tokenized_sent):
    return list(map(lambda x : x[0], filter(lambda x: x[1] == 'NNP', tokenized_sent)))
#checkear set vs lista, hot encoded vs count vectorized para characters

In [10]:
hot_encoded_sentences   = np.zeros((len(sentences), len(token_to_index)))
hot_encoded_characters  = np.zeros((len(sentences), len(characters)))

i = 0
for sentence in sentences:
    words = word_tokenize(sentence)
    for word in words:
        if word in token_to_index:
            hot_encoded_sentences[i][token_to_index[word]] = 1
    tagged  = nltk.pos_tag(words)
    subjects = set(get_subjects_in_sent(tagged))
    for subject in subjects:
            if subject in characters:
                hot_encoded_characters[i][character_to_index[subject.lower()]] = 1
    i += 1
hot_encoded_sentences

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
sorted(hot_encoded_sentences.mean(axis=0), reverse=True)

[1.0,
 0.5552264071096065,
 0.5442234447735929,
 0.5175624206517139,
 0.4358865848497672,
 0.36373254337706307,
 0.23867964451967838,
 0.21900126957257723,
 0.2179432924248836,
 0.21455776555226408,
 0.1743546339399069,
 0.173508252221752,
 0.15467625899280577,
 0.13605586119339821,
 0.13436309775708843,
 0.12928480744815912,
 0.12865002115954297,
 0.12865002115954297,
 0.12738044858231062,
 0.12103258569614897,
 0.10918324164198054,
 0.10495133305120609,
 0.09669911129919594,
 0.08950486669487939,
 0.0876005078290309,
 0.08082945408379179,
 0.0780787134997884,
 0.07405840033855268,
 0.07003808717731697,
 0.06813372831146847,
 0.06686415573423614,
 0.06580617858654253,
 0.06347862886161659,
 0.06305543800253914,
 0.06178586542530681,
 0.059881506559458314,
 0.059669911129919595,
 0.05945831570038087,
 0.058823529411764705,
 0.051206093948370716,
 0.0490901396529835,
 0.04655099449851883,
 0.04295387219636056,
 0.0414727041895895,
 0.03808717731696995,
 0.03702920016927634,
 0.036817604

In [12]:
hot_encoded_sentences.mean()

0.0024616854552500355

In [13]:
hot_encoded_sentences.std()

0.04955426883699817

In [14]:
outliers_right = np.where(hot_encoded_sentences.mean(axis=0) > hot_encoded_sentences.mean()+1.5*hot_encoded_sentences.std())
print(list(map(lambda x: index_to_token[x], outliers_right[0])))

['magic', 'for', 'him', 'his', 'as', 'them', 'are', 'the', 'to', 'is', 'a', 'on', '.', 'it', "'s", 'not', 'he', 'with', 'and', 'she', 'of', 'in', 'her', 'they', 'that', ',', 'but']


In [15]:
for outlier in outliers_right[0]:
    overused_verbs_lowered.append(index_to_token[outlier])

In [16]:
tokens      = set()
sentences   = []
for ep_summary in eps_summary:
    sents = sent_tokenize(ep_summary)
    sentences.extend(sents)
    for sent in sents:
        words = word_tokenize(sent)
        for word in words:
            if not word.lower() in overused_verbs_lowered:
                tokens.add(word.lower())

In [17]:
token_to_index = dict()
index_to_token = dict()
i = 0
for token in tokens:
    token_to_index[token] = i
    index_to_token[i] = token
    i += 1

In [18]:
hot_encoded_sentences   = np.zeros((len(sentences), len(token_to_index)))
hot_encoded_characters  = np.zeros((len(sentences), len(characters)))

i = 0
for sentence in sentences:
    words = word_tokenize(sentence)
    for word in words:
        if word in token_to_index:
            hot_encoded_sentences[i][token_to_index[word]] = 1
    tagged  = nltk.pos_tag(words)
    subjects = set(get_subjects_in_sent(tagged))
    for subject in subjects:
            if subject in characters:
                hot_encoded_characters[i][character_to_index[subject.lower()]] = 1
    i += 1
hot_encoded_sentences

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
sorted(hot_encoded_sentences.mean(axis=0), reverse=True)

[0.07405840033855268,
 0.07003808717731697,
 0.06813372831146847,
 0.06686415573423614,
 0.06580617858654253,
 0.06347862886161659,
 0.06305543800253914,
 0.06178586542530681,
 0.059881506559458314,
 0.059669911129919595,
 0.05945831570038087,
 0.058823529411764705,
 0.051206093948370716,
 0.0490901396529835,
 0.04655099449851883,
 0.04295387219636056,
 0.0414727041895895,
 0.03808717731696995,
 0.03702920016927634,
 0.03681760473973762,
 0.03618281845112146,
 0.03491324587388912,
 0.03491324587388912,
 0.032797291578501905,
 0.03237410071942446,
 0.030681337283114684,
 0.029411764705882353,
 0.02877697841726619,
 0.028565382987727466,
 0.028353787558188744,
 0.027295810410495135,
 0.02496826068556919,
 0.02496826068556919,
 0.02475666525603047,
 0.02454506982649175,
 0.024333474396953027,
 0.024121878967414304,
 0.023910283537875582,
 0.023910283537875582,
 0.02369868810833686,
 0.02369868810833686,
 0.023275497249259414,
 0.022429115531104528,
 0.022217520101565805,
 0.02179432924248

In [20]:
sorted(hot_encoded_sentences.mean(axis=0), reverse=False)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [21]:
outliers_left = np.where(hot_encoded_sentences.mean(axis=0) == 0)
print(list(map(lambda x: index_to_token[x], outliers_left[0])))

['meshu', 'brachium', 'astas', 'rouge', 'illusion', 'junior', 'protobe', 'le', 'slicing', 'needles', 'vida', 'boismortier', 'klaus', 'east', 'kravitz', 'baron', 'talos', 'yagos', 'meanwhile', 'disciples', 'sol', 'no.4', 'barbary', 'sandscreen', 'west', 'silva', 'mantises', 'dorothy', 'enraged', 'karen', 'risacca', 'shidan', 'h', 'zara', 'valkyrie', 'chrono', 'demon-destroyer', 'waterballs', 'broccos', 'vangeance', 'seabed', 'noelles', 'princess-healing', 'lagris', 'heath', 'unopening', 'purgatory', 'vermillion', 'eliza', 'fangs', 'north', 'kiato', 'cornell', 'mariella', 'mereleona', 'thunderclap', 'cradle', 'xerx', 'imitation', 'analyzing', 'valtos', 'abari', 'bulls', 'rotating', 'rebecca', 'binding', 'revchi', 'orcas', 'noelle', 'vetto', 'ringard', 'jamo', 'department', 'yuno', 'degenerate', 'voltia', 'thunderbird', 'secre', 'sally', 'orsi', 'lunettes', 'disciple', 'scarlet', 'banisher', 'mereoleona', 'glamour', 'arrow', 'eternal', 'selection', 'ado', 'feathers', 'breath', 'tormenta',

In [22]:
for outlier in outliers_left[0]:
    overused_verbs_lowered.append(index_to_token[outlier])

In [23]:
tokens      = set()
sentences   = []
for ep_summary in eps_summary:
    sents = sent_tokenize(ep_summary)
    sentences.extend(sents)
    for sent in sents:
        words = word_tokenize(sent)
        for word in words:
            if not word.lower() in overused_verbs_lowered:
                tokens.add(word.lower())

In [24]:
token_to_index = dict()
index_to_token = dict()
i = 0
for token in tokens:
    token_to_index[token] = i
    index_to_token[i] = token
    i += 1

In [25]:
hot_encoded_sentences   = np.zeros((len(sentences), len(token_to_index)))
hot_encoded_characters  = np.zeros((len(sentences), len(characters)))

i = 0
for sentence in sentences:
    words = word_tokenize(sentence)
    for word in words:
        if word in token_to_index:
            hot_encoded_sentences[i][token_to_index[word]] = 1
    tagged  = nltk.pos_tag(words)
    subjects = set(get_subjects_in_sent(tagged))
    for subject in subjects:
            if subject in characters:
                hot_encoded_characters[i][character_to_index[subject.lower()]] = 1
    i += 1
hot_encoded_sentences

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
sorted(hot_encoded_sentences.mean(axis=0), reverse=True)

[0.07405840033855268,
 0.07003808717731697,
 0.06813372831146847,
 0.06686415573423614,
 0.06580617858654253,
 0.06347862886161659,
 0.06305543800253914,
 0.06178586542530681,
 0.059881506559458314,
 0.059669911129919595,
 0.05945831570038087,
 0.058823529411764705,
 0.051206093948370716,
 0.0490901396529835,
 0.04655099449851883,
 0.04295387219636056,
 0.0414727041895895,
 0.03808717731696995,
 0.03702920016927634,
 0.03681760473973762,
 0.03618281845112146,
 0.03491324587388912,
 0.03491324587388912,
 0.032797291578501905,
 0.03237410071942446,
 0.030681337283114684,
 0.029411764705882353,
 0.02877697841726619,
 0.028565382987727466,
 0.028353787558188744,
 0.027295810410495135,
 0.02496826068556919,
 0.02496826068556919,
 0.02475666525603047,
 0.02454506982649175,
 0.024333474396953027,
 0.024121878967414304,
 0.023910283537875582,
 0.023910283537875582,
 0.02369868810833686,
 0.02369868810833686,
 0.023275497249259414,
 0.022429115531104528,
 0.022217520101565805,
 0.02179432924248

In [27]:
sorted(hot_encoded_sentences.mean(axis=0), reverse=False)

[0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,
 0.00021159542953872197,


In [28]:
hot_encoded_sentences.mean()

0.00150822299087389

In [29]:
hot_encoded_sentences.std()

0.03880654911588642

In [30]:
np.where(np.sum(hot_encoded_sentences, axis=1) == 0)

(array([ 951, 1263, 1434, 3950, 4352]),)

In [31]:
hot_encoded_characters

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
hot_encoded_characters.sum(axis=0)

array([   5.,  375.,   23.,   75.,    9.,    3.,   99.,   60.,   29.,
         95.,  141.,   55.,   24.,  148.,   77.,   58.,    9.,  226.,
        305.,  100.,   93.,   45.,  102.,   11.,   43.,    8.,   44.,
         47.,   53., 1209.,   88.,  128.,  153.,   80.,   59.,  155.,
         15.,   37.,   28.,  188.,   78.,   20.,  120.,   75.,  293.])

In [33]:
count_vectorized_sentences  = np.zeros((len(sentences), len(token_to_index)))
count_vectorized_characters = np.zeros((len(sentences), len(characters)))
i = 0
for sentence in sentences:
    words = word_tokenize(sentence)
    for word in words:
        if word in token_to_index:
            count_vectorized_sentences[i][token_to_index[word]] += 1
    tagged  = nltk.pos_tag(words)
    subjects = get_subjects_in_sent(tagged)
    for subject in subjects:
            if subject in characters:
                count_vectorized_characters[i][character_to_index[subject.lower()]] += 1
    i += 1

In [34]:
count_vectorized_sentences[-1]

array([0., 0., 0., ..., 0., 0., 0.])

In [35]:
sentences[-1]

'As Luck lands on a rooftop behind Svenkin, he thanks the Dark Disciple for the fun fight.'

In [36]:
token_to_index["stronger"]

2978

In [37]:
np.where(count_vectorized_sentences > 1)

(array([  12,   18,   21,   21,   25,   25,   25,   28,   29,   33,   36,
          37,   69,   80,  100,  148,  213,  223,  249,  266,  272,  301,
         304,  317,  339,  367,  373,  373,  410,  424,  430,  438,  473,
         474,  475,  501,  501,  510,  551,  605,  605,  640,  649,  652,
         654,  655,  657,  657,  657,  659,  737,  774,  775,  794,  801,
         804,  819,  876,  886,  896,  901,  906,  906,  906,  906,  906,
         906,  906,  906,  929,  944,  965,  968,  985, 1004, 1043, 1045,
        1089, 1131, 1239, 1249, 1266, 1291, 1291, 1292, 1294, 1325, 1334,
        1351, 1354, 1371, 1371, 1374, 1395, 1396, 1396, 1396, 1406, 1417,
        1432, 1457, 1476, 1510, 1534, 1554, 1572, 1572, 1597, 1601, 1677,
        1691, 1707, 1740, 1773, 1780, 1785, 1801, 1804, 1821, 1822, 1840,
        1840, 1846, 1860, 1872, 1876, 1938, 1943, 1958, 1979, 1983, 1983,
        1983, 1988, 2001, 2002, 2004, 2007, 2007, 2008, 2008, 2010, 2014,
        2014, 2018, 2021, 2024, 2032, 

In [38]:
np.where(count_vectorized_characters != hot_encoded_characters)

(array([   6,    6,    7,   10,   14,   16,   43,   47,   50,   50,   65,
          77,   79,   88,  130,  173,  176,  235,  236,  249,  315,  316,
         353,  354,  369,  381,  394,  395,  397,  423,  460,  544,  574,
         602,  647,  649,  649,  654,  659,  667,  704,  733,  737,  740,
         765,  772,  777,  780,  784,  785,  791,  875,  906,  914,  919,
         930, 1077, 1081, 1126, 1131, 1155, 1167, 1172, 1212, 1249, 1295,
        1309, 1317, 1344, 1366, 1396, 1432, 1457, 1461, 1467, 1476, 1481,
        1503, 1503, 1506, 1528, 1580, 1603, 1607, 1637, 1666, 1703, 1744,
        1802, 1809, 1910, 1916, 1972, 1980, 1981, 1982, 1987, 1994, 2003,
        2005, 2009, 2014, 2035, 2051, 2055, 2056, 2058, 2060, 2061, 2067,
        2086, 2088, 2091, 2108, 2137, 2139, 2142, 2144, 2144, 2156, 2157,
        2161, 2162, 2162, 2170, 2178, 2191, 2195, 2197, 2199, 2201, 2206,
        2217, 2228, 2242, 2256, 2261, 2292, 2296, 2301, 2304, 2305, 2314,
        2323, 2334, 2335, 2338, 2362, 

In [39]:
u = 4570
i = 3626
print(count_vectorized_sentences[u][i])
print(hot_encoded_sentences[u][i])
print(index_to_token[i])

0.0
0.0
lessons


In [40]:
import json

In [41]:
def write_json(to_json, filename):
    json_object = json.dumps(to_json)

    with open(filename, "w") as outfile:
        outfile.write(json_object)

In [42]:
write_json(hot_encoded_characters.tolist(),      "4_hot_encoded_characters_black_clover.json")
write_json(hot_encoded_sentences.tolist(),       "4_hot_encoded_sentences_black_clover.json")
write_json(count_vectorized_characters.tolist(), "4_count_vectorized_characters_black_clover.json")
write_json(count_vectorized_sentences.tolist(),  "4_count_vectorized_sentences_black_clover.json")
write_json(index_to_token,                       "4_index_to_token_black_clover.json")
write_json(token_to_index,                       "4_token_to_index_black_clover.json")
write_json(index_to_character,                   "4_index_to_character_black_clover.json")
write_json(character_to_index,                   "4_character_to_index_black_clover.json")